<a href="https://colab.research.google.com/github/araujoroberts/Proyectos/blob/main/Orbita_perturbada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from numpy import sin , cos , pi 

#Subrutinas

**1) Transformacion de coordenadas geodesicas a cartesianas**

In [ ]:
def GC(phi,lam):
  #Semi eje mayor y menor
  a=6.3781370 ; b=6.3567523
  phi=phi*pi/180 ; lam=lam*pi/180

  N = a**2 / ( a**2 * cos(phi)**2 + b**2 * sin(phi)**2 )**0.5
  x = N * cos(lam) * cos(phi) 
  y = N * sin(lam) * cos(phi)
  z = N * (b/a)**2  * sin(phi)

  pos=np.array([ x , y , z ])

  return pos


**2) transformacion de coordenadas cartesianas a geodesicas** 

In [ ]:
def CG(x,y,z):
  #Semi eje mayor y menor
  a=6.3781370 ; b=6.3567523

  # -- Calculo de la longitud --
  
  # Primer cuadrante
  if x>0 and y>0: 
    longitud = math.atan( y / x )

  # Segundo y tercer cuadrante
  if x<0:
    longitud = math.atan( y / x ) + pi 

  # Cuarto cuadrante
  if x>0 and y<0:
    longitud = math.atan( y / x ) + 2*pi 

  phi = longitud

  if phi>=0 and phi<=pi:
	  longitud=phi

  if (phi>pi and phi<2*pi):
	  longitud=-2*pi+phi

  # -- Calculo de la latitud --

  e = (  1 - (b/a)**2  )**0.5
  eps = e**2 / ( 1 - e**2 ) 
  p = ( x**2 + y**2 )**0.5 ; q = math.atan(z*a/p/b)

  latitud = math.atan( (z+eps*b*sin(q)**3) / (p-e**2*a*cos(q)**3))

  # Subrutina que transforma coord's esfericas en terrestres

  
  latitud=latitud*180/pi
  longitud=longitud*180/pi

  latitud=round(latitud,6)
  longitud=round(longitud,6)
  return latitud, longitud



**3) Newton-Raphson para calcular E**

In [ ]:
def NR(M_,e_,E_):
  import numpy as np
  fx  = lambda E_: E_-e_*np.sin(E_)-M_
  dfx = lambda E_: 1-e_*np.cos(E_)
  
  tol=1e-8
  dE=abs(2*tol)
  Eold=E_

  while (dE >= tol):
    Enew=Eold-fx(Eold)/dfx(Eold)
    dE=abs(Enew-Eold)
    Eold=Enew
  return Eold

**4) Subrutina que calcula la posicion de un punto sobre la superficie terrestre al tiempo $t$ debido a su rotacion**

In [ ]:
def W(A,t_):

  # Angulo que rotó la tierra al tiempo t
  theta = lambda t_: 7.2722052e-5*t_

  # Matriz de rotacion al rededor del eje z
  Rz=np.array([ [ cos(theta(dt)) , -sin(theta(dt)) , 0 ],
                [ sin(theta(dt)) ,  cos(theta(dt)) , 0 ],
                [       0         ,         0        , 1 ]  ])

  Ar = Rz @ A

  return Ar

## Propagador $J_2$
Algoritmo empleado para el calculo de orbitas cuando se considera el bulto ecuatorial.

**Algoritmo**

1) Definir los vectores $\vec{r}_1$ y $\vec{r}_2$

2) Obtener el vector de estado $\{\ h_0\ ,\ a_0\ ,\ e_0\ ,\ i_0\ ,\ \Omega_0\ ,\ \omega_0\ \}$ 

3) Calcular $\theta$ transcurrido $\Delta t$

4) Usar $\theta$ para calcular $\vec{r}\ '$ transcurrido $\Delta t$ 

5) Calcular $\dot \Omega $ y $\dot \omega$ 

6) Usar $\dot \Omega $ y $\dot \omega$ para calcular los nuevos $\Omega$ y $\omega$ 

7) Usar $\Omega$ y $\omega$ para convertir $\vec{r}\ '$ a $\vec{r}$

8) Comparar $\vec{r}$ con $\vec{r}_2$ y calcular $\delta \omega$ y $\delta \Omega$

_________________________________________________________

**1) definimos los vectores $\vec{r}_1$, $\vec{r}_2$ y $\Delta t$**

In [ ]:
#Posicion inicial del meteorito y su norma [ Mm ]
r0=np.array([0.50000,-6.50000,4.50000]) 
#r0=np.array([-3.670,-3.870,4.400])
nr0=np.linalg.norm(r0)

#Velocidad inicial del meteorito y su norma [ Mm/s ]
v0=np.array([ 1.80798627e-03 , 2.20380644e-03 , -4.17498856e-05]) 
#v0=np.array([0.00136451, -0.00039703 , 0.00117139])
#v0=np.array([4.7,-7.4,1])*0.001
nv0=np.linalg.norm(v0)

# Tiempo de vuelo [segundos]
dt=8*60 


**2) Calculamos los elementos orbitales**

In [ ]:
# Definimos la base gecentrica
i=np.array([1,0,0])
j=np.array([0,1,0])
k=np.array([0,0,1])

#Calculo de los elementos orbitales { h , i , e , omega0 , w0 , nu0 }
H=np.cross(r0,v0)
h=np.linalg.norm(H) #---------------------- > h


# Calculamos el vector nodo
N=np.cross(k,H)
n=np.linalg.norm(N)

# Calculamos el vector de exentricidad
GM=3.98613e-4
ex = np.cross(v0,H) / GM -  r0 / nr0
e=np.linalg.norm(ex) #------------------------------> e

# Calculamos p y a
p=h**2/GM
a=p/(1-e**2)

# Calculamos los vectores base en el sistema perifocal
ip=ex/e
kp=H/h
jp=np.cross(kp,ip)

# Calculamos la inclinacion del plano orbital
inclinacion=math.acos(H[2]/h) #------------------------------> i


# Calculamos la longitud del nodo acendente
if N[1]>0:
  OMEGA=math.acos(N[0]/n)  
if N[1]<0:          
  OMEGA=-math.acos(N[0]/n)

omega0=OMEGA #-------------------------------------------> omega0

# Calculamos el argumento del perigeo
if ex[-1]>0:
  w=math.acos( np.dot(N,ex) / (n*e) )
if ex[-1]<0: 
  w=-math.acos( np.dot(N,ex) / (n*e) )

w0=w #----------------------------------------------------> w0

# Calculamos el desfase (angulo inicial despecto al foco)
prueba=np.dot(r0,v0)

if prueba > 0 :
  nu0=math.acos(np.dot(ex,r0)/(e*nr0))

if prueba < 0 : #--------------------------------------------------> nu0
  nu0=-math.acos(np.dot(ex,r0)/(e*nr0))

print('Los elementos orbitales iniciales son:')
print('--------------------------------------')
print('h=',h)
print('i=',inclinacion*180/np.pi,'°')
print('e=',e)
print('omega0=',omega0*180/np.pi,'°')
print('w0=',w0*180/np.pi,'°')
print('nu0=',nu0*180/np.pi,'°')

Los elementos orbitales iniciales son:
--------------------------------------
h= 0.01802206200141971
i= 44.50194965550814 °
e= 0.9004950056037042
omega0= -130.21911764389526 °
w0= -130.807743686958 °
nu0= -175.05186372898828 °


**3) Calculamos $\theta$ transcurrido $\Delta t$**

In [ ]:
# Calculamos el periodo de la orbita
T=2*np.pi*a**1.5/math.sqrt(GM) 

print('a=',a)
print('T=',T,'s')

#calculamos q=2pi/T
q=2*np.pi/T

#Calculamos E0
E0=2*math.atan(math.sqrt((1-e)/(1+e))*math.tan(nu0/2))

#Calculamos t0
t0=( E0-e*np.sin(E0) )/q 
print('t0=',round(t0,3),'s')

#calculamos el tf
tf=t0+dt

#Calculamos el numero de vueltas
nv=tf/T

t32=(nv%1)*T

#Calculamos la anomalia media a ese tiempo
M32=q*t32

# E de arranque
if M32 >= np.pi:
  Ep=M32+e/2

if M32 < np.pi:
  Ep=M32-e/2

#Calculamos anomalia excentrica a ese tiempo E32
E32=NR(M32,e,Ep)

#Calculamos nu32
nu32=2*math.atan(math.sqrt((1+e)/(1-e))*math.tan(E32/2))+2*np.pi
print('nu32=',nu32*180/np.pi,'°')

a= 4.30869634622583
T= 2814.640755318885 s
t0= -1093.013 s
nu32= 194.42510739546088 °


**4) Usamos $\theta$ para calcular $\vec{r}\ '$**

In [ ]:
r=p/(1+e*np.cos(nu32))
x=r*np.cos(nu32) ; y=r*np.sin(nu32)
rft=np.array([x,y,0])

**5) Calculamos $\dot \Omega$ y $\dot \omega$**

In [ ]:
#Antes de pasar vec{r} del perifocal al geocentrico debemos corregir las cantidades perturbadas
# omega0 --> omega  &   w0 --> w
J2=1.08263e-3 ; R=6.371

#Para esto primero calculamos ---->  d(omega)/dt   &   dw/dt  
omega_punto=-1.5*math.sqrt(GM)*J2*R**2*np.cos(inclinacion)/( (1-e**2)**2*a**3.5 )
w_punto=-1.5*math.sqrt(GM)*J2*R**2*(2.5*np.sin(inclinacion)**2-2)/( (1-e**2)**2*a**3.5 )

print('Grados/segundo')
print()
print('d(omega)/dt=',omega_punto*180/np.pi,'°/s')
print('dw/dt=',w_punto*180/np.pi,'°/s  ')
print('----------------------')
print('Grados/dia')
print()
print('d(omega)/dt=',omega_punto*180/np.pi*86400,'°/day')
print('dw/dt=',w_punto*180/np.pi*86400,'°/day')


Grados/segundo

d(omega)/dt= -0.009056879406234987 °/s
dw/dt= 0.00979978824048676 °/s  
----------------------
Grados/dia

d(omega)/dt= -782.5143806987029 °/day
dw/dt= 846.701703978056 °/day


**6) Calculamos $\Omega$ y $\omega$**

In [ ]:
#Calculo de omega32 --> omega32 = omega0 + (dw/dt)dt
omega = omega0 + omega_punto*dt 
w = w0 + w_punto*dt

print('Angulos iniciales')
print('oemga0=',omega0*180.0/np.pi,'°')
print('w0=',w0*180.0/np.pi,'°')
print('___________________________')
print('Angulos finales')
print('omega=',omega*180/np.pi,'°')
print('w=',w*180/np.pi,'°')

Angulos iniciales
oemga0= -130.21911764389526 °
w0= -130.807743686958 °
___________________________
Angulos finales
omega= -134.56641975888806 °
w= -126.10384533152437 °


**7.1) Calculamos $\delta \omega$ y $\delta \Omega$**

In [ ]:
# Calculo de dw  &  d(omega) 

dw = abs( w - w0 ) * 180.0 / np.pi    ;    domega = abs( omega - omega0 ) * 180.0 / np.pi  

print('Calculo de d(omega) y dw')
print('---------------------------')
print('d(omega)=',domega,'°')
print('dw=',dw,'°')

Calculo de d(omega) y dw
---------------------------
d(omega)= 4.347302114992805 °
dw= 4.703898355433633 °


**7.2) convertimos $\vec{r}\ '$ $\rightarrow$ $\vec{r}$**

In [ ]:
#Calculo del nuevo vector posicion en el geocentrico
from numpy import cos,sin 

Rw = np.array( [ [  cos(w) , sin(w) , 0 ],
                 [ -sin(w) , cos(w) , 0 ],
                 [    0    ,   0    , 1 ]   ])

i=inclinacion 
Ri = np.array( [ [  1 ,    0    ,   0    ],
                 [  0 ,  cos(i) , sin(i) ],
                 [  0 , -sin(i) , cos(i) ]   ])


Romega = np.array( [ [  cos(omega) , sin(omega) , 0 ],
                     [ -sin(omega) , cos(omega) , 0 ],
                     [     0       ,     0      , 1 ]   ])

Qgp = Rw @ Ri @ Romega  
Qpg = Qgp.T 

# pasamos rf del perifocal al geocentrico:
rf = Qpg @ rft

# Si no hubiera perturbacion el meteorito deberia caer aqui
rwh1=np.array([ 1.28026624, -4.79022783,  4.0005346 ])

#print('Punto de impacto del meteorito sin perturbacion')
#print('rwh1=',rwh1)
#print('-------------------------------------------------')
print('Punto de impacto del meteorito con perturbacion')
print('rf=',rf)
print()
print('Ahora, debemos comparar esto con la posicion de la casa blanca en ese instante')

Punto de impacto del meteorito con perturbacion
rf= [ 1.35681089 -4.6398211   4.14979106]

Ahora, debemos comparar esto con la posicion de la casa blanca en ese instante


**Posicion de la casa blanca al tiempo $t+\Delta t$**

In [ ]:
# Coordenadas de la casa blanca en t0 : (latitud,longitud)=(38.8976763,-77.0387238)
latitud = 38.8976763 ; longitud = -77.0387238
rwh0 = GC(latitud,longitud)
print('La posicion de la casa blanca en t0 es:')
print(rwh0)
print('----------------------------------------')

# Coordenadas de la casa blanca al tiempo t0+dt
rwh=W(rwh0,dt)

print('Posicion de la casa blanca en t0+dt:')
print(rwh)
print()

rwhll=CG(rwh[0],rwh[1],rwh[2])
print('(latitud,longitud): '+str("{0:.6f}".format(rwhll[0]))+',',"{0:.6f}".format(rwhll[1]))


La posicion de la casa blanca en t0 es:
[ 1.11483857 -4.84382997  3.98348271]
----------------------------------------
Posicion de la casa blanca en t0+dt:
[ 1.28320667 -4.80197193  3.98348271]

(latitud,longitud): 38.897676, -75.038724


**Comparamos $\vec{r}_{imp}$ con $\vec{r}_{wh}$**

In [ ]:
print('rimp=',rf)
print('rwh =',rwh)

rimp= [ 1.35681089 -4.6398211   4.14979106]
rwh = [ 1.28320667 -4.80197193  3.98348271]


**Expresamos lo anterior en coordenadas geodesicas**

In [ ]:
# Para poderlas visualizar en el mapa debemos aplicar la antirotacion, esto es Rz(-theta)
theta =  7.2722052e-5*dt

# Matriz de anti-rotacion al rededor del eje z
Rz=np.array([ [ cos(-theta) , -sin(-theta) , 0 ],
              [ sin(-theta) ,  cos(-theta) , 0 ],
              [      0      ,       0      , 1 ]  ])
  
Rf = Rz @ rf 

coordenadas=CG(Rf[0],Rf[1],Rf[2])

print('Coords geodesicas del punto de impacto :',coordenadas)
print('Coords geodesicas de la casa blanca    : (38.8976763,-77.0387238)',)

Coords geodesicas del punto de impacto : (40.834181, -75.699633)
Coords geodesicas de la casa blanca    : (38.8976763,-77.0387238)
